In [1]:
import numpy as np
import pandas as pd


In [12]:
df = pd.read_csv('C:/Workspace/02.DataAnalysis/07.NLP/spam.csv',encoding='latin1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


## 데이터 전처리

In [13]:
del df['Unnamed: 2']
del df['Unnamed: 3']
del df['Unnamed: 4']
df['label'] = df.v1.apply(lambda x: 1 if x == 'spam' else 0)
df.head()

,v1,v2,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
 2   label   5572 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 130.7+ KB


## 전처리

In [14]:
# 중복확인
df.v2.nunique()

5169

In [15]:
# 중복 제거
df.drop_duplicates('v2', keep='first',inplace=True)


In [16]:
# 구둣점 제거
df['content'] = df.v2.str.replace('[^A-Za-z ]','')
df.head()

<ipython-input-16-d443fe1a4ecd>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df.v2.str.replace('[^A-Za-z ]','')


,v1,v2,label,content
0,ham,"Go until jurong point, crazy.. Available only ...",0,Go until jurong point crazy Available only in ...
1,ham,Ok lar... Joking wif u oni...,0,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,Free entry in a wkly comp to win FA Cup final...
3,ham,U dun say so early hor... U c already then say...,0,U dun say so early hor U c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,Nah I dont think he goes to usf he lives aroun...


In [17]:
# 소문자 변환
df['content'] = df.content.apply(lambda x: x.lower())

In [18]:
df.v1.value_counts()

ham     4516
spam     653
Name: v1, dtype: int64

## 훈련/테스트 데이터 셋으로 분리하고 DTM으로 변환

In [19]:
from sklearn.model_selection import train_test_split
str_train, str_test, y_train, y_test = train_test_split(
    df.content, df.label, test_size=0.2,
    stratify=df.label, random_state=2021
)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english', ngram_range=(1,2))
cvect.fit(str_train)

CountVectorizer(ngram_range=(1, 2), stop_words='english')

In [21]:
X_train = cvect.transform(str_train)
X_test = cvect.transform(str_test)

## 훈련/예측/평가

In [22]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_test)

In [23]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.9777562862669246